# IT market analysis

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import csv

In [4]:
import pandas as pd

In [5]:
import time

In [6]:
def extract_job_data(soup, mode):
    """Funckja wyciągająca wszystkie interesujące nas dane 
        z ofert pracy wyświetlonych na indeed"""
    
    jobs_data= []    # Lista wszystkich ofert
    
    # Pętla po ofertach na aktualnie analizowanej stronie
    for div in soup.find_all(name= "div", attrs={"class":"row"}):
        
        job = []    # Lista z danymi konkretnej oferty
        
        # Wyciągamy nazwę pracy
        job_title = div.find_all(name="a", attrs={"data-tn-element":"jobTitle"})
        if len(job_title) > 0:
            titles = ""
            for a in job_title:
                if titles != "":
                    titles += " | "
                titles += a["title"]
            job.append(titles)
        else:
            job.append("Nothing found")
            
        
        # Wyciągamy nazwę firmy
        company = div.find_all(name="span", attrs={"class":"company"})
        companies = ""
        if len(company) > 0:
            for b in company:
                if companies != "":
                    companies += " | " 
                companies += b.text.strip()
            job.append(companies)
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            if len(sec_try) == 0:
                job.append("Nothing found")
            else:
                for span in sec_try:
                    if companies != "":
                        companies += " | " 
                    companies += span.text.strip()
                job.append(companies)
        
        
        # Wyciągamylokazlizację firmy, ocenę firmy i ilość oddanych głosów
        locationDiv = div.find("div", attrs={"class":"sjcl"})
        spans = locationDiv.find_all("span", attrs={"class": "location"})
        
        if len(spans) == 0:
            job.append("Nothing found")
        else:
            locations = ""
            for span in spans:
                if locations != "":
                    locations += " | "
                locations += span.text.strip()
            job.append(locations)
            
        rating = locationDiv.find("span", attrs={"class": "rating"})
        views = locationDiv.find("span", attrs={"class": "slNoUnderline"})

        if rating:
            rating = format(float(rating["style"].replace("width:", "").replace("px","")) / 60 * 5, '.2f')
        else:
            rating = "Nothing found"

        if views:
            var = views.text.replace(",", "").replace("\xa0", "")
            views = int(var.split(" ")[0].strip())
        else:
            views = "Nothing found"
            
        job.append(rating)
        job.append(views)
        
        # Wyciągamy dane o zarobkach
        salaryDiv = div.find("div", attrs={"class":"salarySnippet"})
        try:
            job.append(salaryDiv.find("span", attrs={"class":"salary no-wrap"}).text.replace('\n','').strip())
        except:
            try:
                div_two = salaryDiv.find(name="span", attrs={"class":"salary-est-similar no-wrap"})
                job.append(div_two.text.replace('\n','').strip())
            except:
                job.append("Nothing found")
                
                
        # Wyciągamy opis stanowiska
        try:
            job.append(div.find("div", attrs={"class":"summary"}).text.replace('\n','').strip())
        except:
            job.append("Nothing found")
            
        
        # Sprawdzamy kiedy oferta została dodana
        dateDiv = div.find("div", attrs={"class":"jobsearch-SerpJobCard-footerActions"})       
        try:
            job.append(dateDiv.find("span", attrs={"class": "date"}).text)
        except:
            job.append("Nothing found") 
    
    
        # Wyciągamy link do strony z ofertą pracy
        if mode == 'pl':
            newPageDiv = "https://pl.indeed.com" + div.find("div", attrs={"class":"title"}).a["href"] + "rom=vj&pos=bottom"
        elif mode == 'usa':
            newPageDiv = "https://indeed.com" + div.find("div", attrs={"class":"title"}).a["href"] + "rom=vj&pos=bottom"
        else:
            raise Exception("Wrong mode argument") 
        
        if newPageDiv == "https://pl.indeed.comrom=vj&pos=bottom" or newPageDiv == "https://indeed.comrom=vj&pos=bottom":
            newPageDiv = "Nothing found"
            
        job.append(newPageDiv)
        
        # Dodajemy ofertę do listy ofert
        jobs_data.append(job)
        
    return jobs_data

In [7]:
def prepareSalary(salary, mode):
    """Funkcja pomagająca zapisać informacje 
    o wyszukiwanych przez nas zarobkach w linku"""
     
    if salary == None:
        return ""
    elif mode == "pl":
        var = ""
    elif mode == "usa":
        var = "%24"
    else:
        raise Exception("Wrong mode argument") 
        
    if len(str(salary)) <= 3:
        strSalary = var + "%2C" + "0"*(3 - len(str(salary))) + str(salary)
    else:
        strSalary = var + str(salary)[:-3] + "%2C" + str(salary)[-3:]
        
    return strSalary

In [8]:
usa_city_set = ['New York','Chicago','San Francisco', 'Austin', 'Seattle',
            'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh',
            'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington DC', 'Boulder']

pl_city_set = ['Wrocław, dolnośląskie', 'Warszawa, mazowieckie', 'Kraków, małopolskie', 'Poznań, wielkopolskie',
              'Łódź, łódzkie', 'Szczecin, zachodniopomorskie', 'Gdańsk, pomorskie', 'Katowice, śląskie',
               'Rzeszów, podkarpackie', 'Bydgoszcz, kujawsko-pomorskie', 'Toruń, kujawsko-pomorskie', 
               'Legnica, dolnośląskie', 'Opole, opolskie']

columns = ["City", "Job Title", "Company Name", "Location", "Rating", "Views", "Salary", "Summary", "Date", "Link"]


In [9]:
def make_clickable(url):
    """Funkcja pozwalająca na wyświetlenie obiektu dataframe 
        z linkiem do strony z ofertą"""
    
    return '<a target="_blank" href="{}">{}</a>'.format(url, "Link")


In [10]:
def scrappingFunction(job, cities, columnsNames, maxResults, mode, fileName, salary = None,):
    """Funkcja przeszukująca strony według kryteriów podnaych w argumentach. 
        Zwaraca obiekt dataframe ze wszystkimi danymi."""
    if mode == 'pl':
        url = 'http://pl.indeed.com/praca?q=' + job.replace(' ', '+') + '+' + prepareSalary(salary, mode) + '&l='
    elif mode == 'usa':
        url = 'http://indeed.com/praca?q=' + job.replace(' ', '+') + '+' + prepareSalary(salary, mode) + '&l='
    else:
        raise Exception("Wrong mode argument") 
    sample_df = pd.DataFrame(columns = columnsNames)


    for city in cities:
        for start in range(0, maxResults, 10):
            num = (len(sample_df) + 1) 
            page = requests.get(url + city.replace(',', '%2C').replace(' ', '+') + '&start=' + str(start))
            time.sleep(1)  #ensuring at least 1 second between page grabs
            soup = BeautifulSoup(page.text, 'lxml')

            jobs_data = extract_job_data(soup, mode)
            
            for i in range(len(jobs_data)):
                sample_df.loc[num + i] = [city] +  jobs_data[i]
                
    sample_df = sample_df.drop_duplicates()
    #saving sample_df as a local csv file — define your own local path to save contents 
    sample_df.to_csv( fileName+ '.csv', encoding='utf-8')
    return sample_df

In [11]:
usa_data_scientist = scrappingFunction('data scientist', usa_city_set, columns, 100, 'usa', 'usa_data_scientist')
usa_data_scientist_10 = scrappingFunction('data scientist', usa_city_set, columns, 100, 'usa', 'usa_data_scientist_10', 10000)
usa_data_scientist_20 = scrappingFunction('data scientist', usa_city_set, columns, 100, 'usa', 'usa_data_scientist_20', 20000)

In [12]:
pl_data_scientist = scrappingFunction('data scientist', pl_city_set, columns, 100, 'pl', 'pl_data_scientist')
pl_data_scientist_10 = scrappingFunction('data scientist', pl_city_set, columns, 100, 'pl', 'pl_data_scientist_10', 10000)
pl_data_scientist_20 = scrappingFunction('data scientist', pl_city_set, columns, 100, 'pl', 'pl_data_scientist_20', 20000)

In [13]:
usa_python = scrappingFunction('python', usa_city_set, columns, 100, 'usa', 'usa_python' )
usa_python_10 = scrappingFunction('python', usa_city_set, columns, 100, 'usa', 'usa_python_10', 10000)
usa_python_20 = scrappingFunction('python', usa_city_set, columns, 100, 'usa', 'usa_python_20', 20000)

In [14]:
pl_python = scrappingFunction('python', pl_city_set, columns, 100, 'pl', 'pl_python')
pl_python_10 = scrappingFunction('python', pl_city_set, columns, 100, 'pl', 'pl_python_10', 10000)
pl_python_20 = scrappingFunction('python', pl_city_set, columns, 100, 'pl', 'pl_python_20', 20000)

In [15]:
usa_python_intern = scrappingFunction('python intern', usa_city_set, columns, 100, 'usa', 'usa_python_intern')
pl_python_intern = scrappingFunction('python staż', pl_city_set, columns, 100, 'pl', 'pl_python_intern')

## Data Science w USA

### Bez podania oczekiwanych zarobków

In [20]:
usa_data_scientist[usa_data_scientist["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
270,San Francisco,Staff Data Scientist,Walmart,"San Bruno, CA 94066",3.55,179534,Nothing found,"We use advanced statistical modeling, optimization techniques, machine learning, and data mining underneath it all to analyze all this data on top of Hadoop...",16 days ago,Link
275,San Francisco,Data Scientist 1274618BR,Walmart,"San Bruno, CA 94066",3.55,179534,Nothing found,"Data Scientists are responsible for researching new trends in the industry and utilizing up-to-date technology (for example, HBase, MapReduce, LAPack, Gurobi)...",30+ days ago,Link
342,Austin,Principal Data Scientist - BlackLocus,The Home Depot,"Austin, TX 73301 (St Edwards area)",3.65,46772,Nothing found,35%-Designs and develops solutions that leverage statistical analytics and optimization to derive business insights. Years of Relevant Work Experience:....,30+ days ago,Link
328,Austin,Data Scientist - BlackLocus,The Home Depot,"Austin, TX 73301 (St Edwards area)",3.65,46772,Nothing found,"Data Scientists are expected to contribute to projects as a hands-on, expert-level resource, capable of synthesizing the business requirements, nuances of the...",30+ days ago,Link
709,Atlanta,DATA SCIENTIST - SUPPLY CHAIN,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46772,Nothing found,"Data Scientists are responsible for organizing, analyzing, and then sharing insights gleaned from data. The Data Scientist will be integral part of the Channel...",3 days ago,Link


### 10 tysięcy dolarów oczekiwanych zarobków

In [21]:
usa_data_scientist_10[usa_data_scientist_10["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
317,Austin,Data Scientist - BlackLocus,The Home Depot,"Austin, TX 73301 (St Edwards area)",3.65,46772,Nothing found,"Experience in data mining, data analysis and predictive modeling. A BlackLocus Data Scientist solves difficult business problems with advanced statistical...",30+ days ago,Link
717,Atlanta,"Data Scientist, Online",The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46772,Nothing found,Data Scientist. Strong communication and data presentation skills. Insights from data to solve business problems....,30+ days ago,Link
784,Atlanta,DATA SCIENTIST - SUPPLY CHAIN,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46772,Nothing found,"Data Scientists are responsible for organizing, analyzing, and then sharing insights gleaned from data. A Data Scientist leverages their technical abilities to...",3 days ago,Link
738,Atlanta,"Data Scientist, Online",The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46772,Nothing found,"Position Purpose:. At HomeDepot.com, our award-winning e-Commerce team is blazing a trail of retail shopping innovation by leveraging cutting-edge technology...",30+ days ago,Link
729,Atlanta,"Associate Data Scientist, Online",The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46772,Nothing found,Experience with large-scale data analysis and a demonstrated ability to identify key insights from data to solve business problems....,30+ days ago,Link


### 20 tysięcy dolarów oczekiwanych zarobków

In [22]:
usa_data_scientist_20[usa_data_scientist_20["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
709,Atlanta,Data Scientist,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46774,Nothing found,Data Scientist. Strong communication and data presentation skills. Insights from data to solve business problems....,30+ days ago,Link
705,Atlanta,DATA SCIENTIST - SUPPLY CHAIN,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46774,Nothing found,"Data Scientists are responsible for organizing, analyzing, and then sharing insights gleaned from data. A Data Scientist leverages their technical abilities to...",3 days ago,Link
771,Atlanta,"Senior Data Scientist, Online",The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46774,Nothing found,"At HomeDepot.com, our award-winning e-Commerce team is blazing a trail of retail shopping innovation by leveraging cutting-edge technology with our 2,200+ store...",30+ days ago,Link
395,Austin,Senior Data Engineer - BlackLocus,The Home Depot,"Austin, TX 73301 (St Edwards area)",3.65,46774,Nothing found,"Experience with SQL, NoSQL databases, relational databases, and query authoring. 8+ years of previous related work experience....",30+ days ago,Link
708,Atlanta,DATA ANALYST - SUPPLY CHAIN ANALYTICS,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46774,Nothing found,"Operates under the supervision and mentorship of more experienced managers and data scientists. 1-3 years work experience in data mining, statistical analysis,...",4 days ago,Link


## Data Science w Polsce

### Bez podania oczekiwanych zarobków

In [23]:
pl_data_scientist[pl_data_scientist["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
459,"Gdańsk, pomorskie",TTS Language Engineer - Portuguese,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"QA of TTS voices and linguistic data. As a TTS Language Engineer you will work alongside Software Developers, Audio Engineers, Research Scientist and Program...",30+ dni temu,Link
458,"Gdańsk, pomorskie","Machine Learning Scientist, TTS Research",AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"Computer Science fundamentals in algorithm design, complexity analysis, data structures, problem solving and diagnosis....",30+ dni temu,Link
456,"Gdańsk, pomorskie","Deep Learning Scientist, TTS Research",AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"Computer Science fundamentals in algorithm design, complexity analysis, data structures, problem solving and diagnosis....",30+ dni temu,Link
140,"Warszawa, mazowieckie",Data Scientist,AWS EMEA SARL (Poland Branch),"Warszawa, mazowieckie",3.55,34836,Nothing found,Interested in creating new state-of-the-art solutions using Machine Learning and Data Mining techniques on Terabytes of Data?...,30+ dni temu,Link
158,"Warszawa, mazowieckie",Applied Intelligence Big Data Scientist,Accenture,"Warszawa, mazowieckie",4.25,16644,Nothing found,"Use Big Data technologies to improve Client businesses by combining standard and advanced data processing such as information extraction from unstructured data,...",30+ dni temu,Link


### 10 tysięcy złotych oczekiwanych zarobków

In [48]:
pl_data_scientist_10[(pl_data_scientist_10["Views"] != "Nothing found") & (pl_data_scientist_10["Job Title"] != "Pracownik restauracji McDonalds")].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
1545,"Legnica, dolnośląskie",Lekka Praca Produkcyjna - Bezpłatny Dojazd - Wysokie Stawki,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",12 dni temu,Link
1547,"Legnica, dolnośląskie",Pracownik Produkcji Automotive,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",27 dni temu,Link
750,"Łódź, łódzkie",Asystent ds. Rekrutacji,DHL Supply Chain,"Tuszyn, łódzkie",3.65,14677,Nothing found,"W roku 2018 zatrudniliśmy ponad 1 000 osób a 70% kadry kierowniczej wyłaniamy w ramach wewnętrznych promocji,....",30+ dni temu,Link
809,"Szczecin, zachodniopomorskie",Glasbud,Adecco,"Szczecin, zachodniopomorskie",3.65,14490,Nothing found,Atrakcyjne wynagrodzenie min. 6 000 Brutto/ miesięcznie. Adecco Poland Sp. Z o.o. Należy do międzynarodowej korporacji The Adecco Group – światowego lidera...,25 dni temu,Link
810,"Szczecin, zachodniopomorskie",Magazynier,Adecco,"Szczecin, zachodniopomorskie",3.65,14490,Nothing found,W 2015 roku pracę dzięki Adecco Poland znalazło ponad 60 000 osób. Adecco Poland Sp. Z o.o. Należy do międzynarodowej korporacji The Adecco Group – światowego...,16 dni temu,Link


### 20 tysięcy złotych oczekiwanych zarobków

In [49]:
pl_data_scientist_20[(pl_data_scientist_20["Views"] != "Nothing found") & (pl_data_scientist_20["Job Title"] != "Pracownik restauracji McDonalds")].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
1542,"Legnica, dolnośląskie",Lekka Praca Produkcyjna - Bezpłatny Dojazd - Wysokie Stawki,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",12 dni temu,Link
1544,"Legnica, dolnośląskie",Pracownik Produkcji Automotive,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",27 dni temu,Link
746,"Łódź, łódzkie",Asystent ds. Rekrutacji,DHL Supply Chain,"Tuszyn, łódzkie",3.65,14677,Nothing found,"W roku 2018 zatrudniliśmy ponad 1 000 osób a 70% kadry kierowniczej wyłaniamy w ramach wewnętrznych promocji,....",30+ dni temu,Link
1337,"Bydgoszcz, kujawsko-pomorskie",Praca przy wykładaniu towaru,Adecco,"Bydgoszcz, kujawsko-pomorskie",3.65,14490,Nothing found,W 2016 roku pracę dzięki Adecco Poland znalazło ponad 63 000 osób. Adecco Poland Sp. Z o.o. Należy do międzynarodowej korporacji The Adecco Group – światowego...,2 dni temu,Link
813,"Szczecin, zachodniopomorskie",Praca od ZARAZ -Spawacz MIG/MAG delegacja krajowa Płock,Adecco,"Szczecin, zachodniopomorskie",3.65,14490,Nothing found,Atrakcyjne wynagrodzenie min. 6 000 Brutto/ miesięcznie. Adecco Poland Sp. Z o.o. Należy do międzynarodowej korporacji The Adecco Group – światowego lidera...,25 dni temu,Link


## Programista Python w USA

### Bez podania oczekiwanych zarobków

In [26]:
usa_python[usa_python["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
1382,Houston,Business Intelligence Analyst,The Home Depot,"Houston, TX 77001",3.65,46774,Nothing found,POSITION PURPOSE. Want to be a key member of the centralized Business Intelligence team within The Home Depot's hub of innovation and one of the fastest growing...,29 days ago,Link
1405,Miami,Admin- Splunk,SUBWAY,"Miami, FL",3.60,38033,Nothing found,"Splunk Certified Power User, Splunk Certified Admin is a plus. The Splunk engineer will support the full system engineering life-cycle, including requirements...",30+ days ago,Link
29,New York,Solutions Consultant I,AT&T,"New York, NY",3.65,36603,Nothing found,"Proficiency with data analysis and manipulation in Excel and a demonstrable passion for learning new technologies including digital media buying, MySQL, APIs or...",10 days ago,Link
753,Atlanta,Analyst – GCSC Strategy,AT&T,"Atlanta, GA",3.65,36603,Nothing found,"Python, and/or other programming languages. As a member of the Global Connections and Supply Chain (GCSC)....",30+ days ago,Link
4,New York,Product Support Specialist I,AT&T,"New York, NY",3.65,36603,Nothing found,"Knowledge of SQL, HTML, Python and/or data analysis preferred. The Product Support Specialist maintains a dedicated focus on ensuring that customers are...",5 days ago,Link


### 10 tysięcy dolarów oczekiwanych zarobków

In [27]:
usa_python_10[usa_python_10["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
269,San Francisco,"Analyst, Operations - Customer Care",Walmart,"San Bruno, CA 94066",3.55,179537,Nothing found,"Demonstrates up-to-date expertise in Walmart.com operational data analysis and applies this to the development, execution, and improvement of action plans....",18 days ago,Link
170,Chicago,"Manager, Customer Data Strategy",McDonald's Corporate,"Chicago, IL",3.50,158116,Nothing found,You are proficient in SQL and R or Python. Do you want a career and not just a job? Are you ready to make a difference and not just make a living?...,30+ days ago,Link
1447,Miami,Admin- Splunk,SUBWAY,"Miami, FL",3.60,38033,Nothing found,"Job Details. Hungry for a fresh, new career? Look no further because one of the world’s most iconic brands can help you get there....",30+ days ago,Link
24,New York,Solutions Consultant I,AT&T,"New York, NY",3.65,36603,Nothing found,"Proficiency with data analysis and manipulation in Excel and a demonstrable passion for learning new technologies including digital media buying, MySQL, APIs or...",10 days ago,Link
27,New York,Product Support Specialist I,AT&T,"New York, NY",3.65,36603,Nothing found,"Knowledge of SQL, HTML, Python and/or data analysis preferred. The Product Support Specialist maintains a dedicated focus on ensuring that customers are...",5 days ago,Link


### 20 tysięcy dolarów oczekiwanych zarobków

In [28]:
usa_python_20[usa_python_20["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
730,Atlanta,DATA SCIENTIST - SUPPLY CHAIN,The Home Depot,"Atlanta, GA 30301 (Buckhead area)",3.65,46774,Nothing found,"A thorough understanding of SQL and intermediate knowledge of one other scripting/compiled/statistical programming language (VBA, Python, R, SAS, .NET, C++,...",3 days ago,Link
1377,Houston,Business Intelligence Analyst,The Home Depot,"Houston, TX 77001",3.65,46774,Nothing found,POSITION PURPOSE. Want to be a key member of the centralized Business Intelligence team within The Home Depot's hub of innovation and one of the fastest growing...,29 days ago,Link
1445,Miami,Admin- Splunk,SUBWAY,"Miami, FL",3.60,38033,Nothing found,"Splunk Certified Power User, Splunk Certified Admin is a plus. The Splunk engineer will support the full system engineering life-cycle, including requirements...",30+ days ago,Link
868,Dallas,Professional-Advanced Analytics,AT&T,"Plano, TX",3.65,36603,Nothing found,Build predictive models using R and Python and statistically test the. Building predictive models using languages such as R and Python and perform statistical...,30+ days ago,Link
756,Atlanta,Analyst – GCSC Strategy,AT&T,"Atlanta, GA",3.65,36603,Nothing found,"Python, and/or other programming languages. As a member of the Global Connections and Supply Chain (GCSC)....",30+ days ago,Link


## Programista Python w Polsce

### Bez podania oczekiwanych zarobków

In [29]:
pl_python[pl_python["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
979,"Gdańsk, pomorskie",Workforce Analyst,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,VBA Python scripting skills is a plus. Bachelor’s degree or equivalent experience. 3+ years of Call Centre or similar experience related to a solid...,30+ dni temu,Link
1000,"Gdańsk, pomorskie",TTS Language Engineer - Portuguese,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"Perl, Python). Solid knowledge of phonetics/phonology. Native-level proficiency in the target language. Solid knowledge of regular expressions....",30+ dni temu,Link
1008,"Gdańsk, pomorskie",Technical Subject Matter Expert,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"Practical knowledge of any programming language (Java, Python, VBA) will be an advantage. Basic Qualifications....",30+ dni temu,Link
1028,"Gdańsk, pomorskie",Data Engineer,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,"Bachelor's degree in Computer Science, Mathematics, Statistics, Finance, related technical field, or equivalent work experience. Hadoop, Apache Spark, etc....",30+ dni temu,Link
975,"Gdańsk, pomorskie",Software Development Manager - Alexa,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,BS/MS in Computer Science or related technical discipline. 5+ years of industry experience. 3+ years of experience in managing software development teams....,4 dni temu,Link


### 10 tysięcy złotych oczekiwanych zarobków

In [44]:
pl_python_10[(pl_python_10["Views"] != "Nothing found") & (pl_python_10["Job Title"] != "Pracownik restauracji McDonalds")].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
922,"Gdańsk, pomorskie",Real Time Analyst,AMZN Dev Cntr Poland sp. z.o.o,"Gdańsk, pomorskie",3.55,34836,Nothing found,VBA Python scripting skills is a plus. Bachelor’s degree. 1+ years of Real Time Monitoring (RTM) and/or 1+ years of Scheduling experience in a Call Center or...,30+ dni temu,Link
768,"Łódź, łódzkie",Praktykant - Java Developer,Accenture,"Łódź, łódzkie",4.25,16644,Nothing found,"C/C++, PHP, C#, Python czy Java. Siennej 39, NIP 526-00-15-900 (Administrator danych), zgodnie z ustawą z dnia 10 maja 2018 r. Praktykant – Java Developer....",30+ dni temu,Link
1084,"Katowice, śląskie",Starszy Programista Talend,Accenture,"Katowice, śląskie",4.25,16644,Nothing found,"Scala, Python, Java, R; Siennej 39, NIP 526-00-15-900 (Administrator danych), zgodnie z ustawą z dnia 10 maja 2018 r. Starszy Programista Talend....",30+ dni temu,Link
1039,"Katowice, śląskie",Back-end Developer,Accenture,"Katowice, śląskie",4.25,16644,Nothing found,"Back-end Developer (Java, .NET, PHP, Python). Sienna 39, NIP 526-00-15-900 (Data Controller), in accordance with the Act of May 10, 2018 on the Protection of...",30+ dni temu,Link
786,"Łódź, łódzkie",Starszy Programista Talend (Lodz),Accenture,"Łódź, łódzkie",4.25,16644,Nothing found,"Scala, Python, Java, R. Sienna 39, NIP 526-00-15-900 (Data Controller), in accordance with the Act of May 10, 2018 on the Protection of Personal Data (Journal...",30+ dni temu,Link


### 20 tysięcy złotych oczekiwanych zarobków

In [47]:
pl_python_20[(pl_python_20["Views"] != "Nothing found") & (pl_python_20["Job Title"] != "Pracownik restauracji McDonalds")].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
376,"Kraków, małopolskie",IBM Poland Internship - Quality Assurance Student,IBM,"Kraków, małopolskie",3.70,25943,Nothing found,"Ability to create and reuse advanced shells (Bash, Perl, Python). We are looking for students available for at least 20 hours per week....",30+ dni temu,Link
356,"Kraków, małopolskie",IBM Poland Internship - Support Student,IBM,"Kraków, małopolskie",3.70,25943,Nothing found,"Ability to create and reuse advanced shells (Bash, Perl, Python). We are looking for students available for at least 20 hours per week....",30+ dni temu,Link
1513,"Legnica, dolnośląskie",Pracownik Produkcji Automotive,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",27 dni temu,Link
1511,"Legnica, dolnośląskie",Lekka Praca Produkcyjna - Bezpłatny Dojazd - Wysokie Stawki,Manpower,"Nowa Wieś Legnicka, dolnośląskie",3.65,15332,Nothing found,"Otrzymasz korzystne wynagrodzenie podstawowe, a ponadto podniesiesz je dzięki wielu benefitom finansowym - docelowo nawet 4 000 zł brutto....",12 dni temu,Link
756,"Łódź, łódzkie",Asystent ds. Rekrutacji,DHL Supply Chain,"Tuszyn, łódzkie",3.65,14677,Nothing found,"W roku 2018 zatrudniliśmy ponad 1 000 osób a 70% kadry kierowniczej wyłaniamy w ramach wewnętrznych promocji,....",30+ dni temu,Link


## Stażysta Python 

### USA

In [31]:
usa_python_intern[usa_python_intern["Views"] != "Nothing found"].sort_values(by = "Views", ascending = False).head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
1324,Washington DC,2019 Systems Development Fall Intern,"Amazon Web Services, Inc.","Herndon, VA",3.55,34838,Nothing found,"Demonstrated proficiency with scripting languages such as Bash, Python, C, C++, Java or Ruby. You should be an engineer who is focused on developing internal...",10 days ago,Link
428,Seattle,Applied Scientist Fall/Winter 2019 Internship - Natural Language Processing,"Amazon.com Services, Inc.","Seattle, WA",3.55,34838,Nothing found,"Skills with Python, C++, Java or other programming language, as well as with R, MATLAB or similar scripting language. Master’s or Ph.D....",30+ days ago,Link
444,Seattle,Graduate Research Applied Scientist,"Amazon.com Services, Inc.","Seattle, WA",3.55,34838,Nothing found,"Machine Learning / Deep Learning /AI. Commercial speech products or government speech projects). An understanding of machine learning, algorithms and...",30+ days ago,Link
443,Seattle,Software Development Engineer,"Amazon.com Services, Inc.","Seattle, WA",3.55,34838,Nothing found,Computer Science fundamentals in data structures. Bachelor’s Degree in Computer Science or related field. We are looking for innovative software engineers who...,30+ days ago,Link
434,Seattle,Graduate Research Applied Scientist,"Amazon.com Services, Inc.","Seattle, WA",3.55,34838,Nothing found,"Skills with Java, C++, or other programming language, as well as with R, MATLAB, Python or similar scripting language. Currently enrolled in a Masters or Ph.D....",30+ days ago,Link


### Polska

In [34]:
pl_python_intern.head().style.format({'Link': make_clickable})

,City,Job Title,Company Name,Location,Rating,Views,Salary,Summary,Date,Link
1,"Wrocław, dolnośląskie",specjalista ds. wsparcia technicznego (JUNIOR lub STAŻ),recruit.pl,"Wrocław, dolnośląskie",Nothing found,Nothing found,Nothing found,"Wsparcia technicznego (JUNIOR lub STAŻ). Windows Server, AWS, Office 365, programowanie PHP lub SQL lub Python, Visual Basic administracja). Specjalista ds....",23 dni temu,Link
2,"Wrocław, dolnośląskie",Stażysta Programista,gumtree.pl,"Wrocław, dolnośląskie",Nothing found,Nothing found,Nothing found,"Potrafisz sprawnie programować w minimum jednym języku (PHP, Java, C/C++, Kotlin, Python, JavaScript). Jeżeli studiujesz i chcesz jednocześnie aktywnie budować...",30 dni temu,Link
21,"Warszawa, mazowieckie",Data Science Konsultant ( SAS / Python / R ),BD Polska sp. z o. o.,Nothing found,Nothing found,Nothing found,Nothing found,"Doświadczenie w SAS / Python / R. Chcesz tworzyć innowacje, rozwiązaywać wyzwania analityczne z użyciem zaawansowanej analizy danych? BD Polska sp....",Nothing found,"<a target=""_blank"" href=""https://pl.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AMB-cy4UzjB16h_5phwYz7q6cNwuWzTU6HKTuPrlGq1j51e0aNtsHNn4wvhF4MCSBxbA5urOzEwmTeJpZpilGwfguWdmVNprw38LwQDLKFTcwp19RgPqJtzOtwl8U9nytBM8bFYEeuZ7b0ttQG6njzLM1ZgLXaUS2u9q2PhF7JvBeUmgJtU__fjvfO3hw40uPONtn4mgxJHJw9HcofcVqtBPFAtBgdYELkhAkrVZBR1yu_pt3d9uHjzP3zFUHSN5k03Leyo91omfpy_dIb24AB0SpNQlYxaMmn3f4m7d8w1yZjFss3brWpXECuRHi42Th52rEtV7GyWeaIWxHXXcSj_85QzcdgUKH82OMMh_QrAb8R3wyZQ6F_k0-kn4x9dauTbNs02tckx4wCyr3gDID0H0d0JDnpW8devmf4aXcKwlXCgbirgU5u1-9oM-l4ROBZCbPZLMbUjI2j3HEvGFjK&p=0&fvj=1&vjs=3rom=vj&pos=bottom"">Link"
22,"Warszawa, mazowieckie",Programista Python,K2 Internet,"Warszawa, mazowieckie",Nothing found,Nothing found,Nothing found,"Ekstra urlop za staż. Praktycznej umiejętności programowania w języku Python,. Poszukujemy osoby, która wesprze nasz zespół jako Programista Python....",30+ dni temu,Link
23,"Warszawa, mazowieckie",Data Science Konsultant ( SAS / Python / R ),BD Polska sp. z o. o.,"Warszawa, mazowieckie",Nothing found,Nothing found,Nothing found,"Doświadczenie w SAS / Python / R. Chcesz tworzyć innowacje, rozwiązaywać wyzwania analityczne z użyciem zaawansowanej analizy danych? BD Polska sp....",13 dni temu,Link
